Fachprojekt Bioinformatik 17/18 von
Niclas Joswig und Thomas Schlütter

In [1]:
import pandas as pd
import numpy as np
import bokeh.palettes as Pal
from bokeh.io import show
from bokeh.models.widgets import Select
from bokeh.server.server import Server
from bokeh.application import Application
from bokeh.application.handlers.function import FunctionHandler
from bokeh.plotting import*
from bokeh.models.widgets import CheckboxButtonGroup,RadioButtonGroup, Button 
from bokeh.models import Toggle 
from bokeh.layouts import column, row, widgetbox
from bokeh.models.widgets.inputs import *
np.set_printoptions(threshold=np.inf)
pd.set_option("display.max_columns",None)

In [2]:
gen_nr = 23323
PORT = 8000

#Init of Data
classes = pd.read_csv("classes.csv", sep="\s+")
normalized_data = pd.read_csv("callithrix-transcriptome-normalized.csv", sep=" ", nrows=gen_nr)
raw_data = pd.read_csv( "callithrix-transcriptome-raw.csv" , sep=" ", nrows=gen_nr)

normalized_data = normalized_data.reindex(columns=['gene',
 'DPZ-ESC-0-p83',
 'DPZ-ESC-0-p91',
 'DPZ-ESC-0-p94',
 'DPZ-ESC-0-p96',
 'DPZ-ESC-1-p51',
 'DPZ-ESC-2-p36',
 'DPZ-ESC-2-p37',
 'DPZ-ESC-3-p37',
 'DPZ-ESC-3-p41',
 'DPZ-ESC-4-p30',
 'DPZ-ESC-4-p33',
 'UKE-ESC-0-p98',
 'DPZ-iPSC-1-p18',
 'DPZ-iPSC-1-p19',
 'DPZ-iPSC-1-p22',
 'DPZ-iPSC-1-p24',
 'DPZ-Fib-0-p06',
 'DPZ-Fib-0-p02',
 'UKE-Fib-0-p03',
 'UKE-piPSC-p05-CM',
 'UKE-piPSC-p05-LF',
 'UKE-piPSC-p05-EL',
 'UKE-piPSC-p05-LC'
 ])

esc=['DPZ-ESC-0-p83', 'DPZ-ESC-0-p91', 'DPZ-ESC-0-p94','DPZ-ESC-0-p96',
     'DPZ-ESC-1-p51','DPZ-ESC-2-p36','DPZ-ESC-2-p37','DPZ-ESC-3-p37',
     'DPZ-ESC-3-p41','DPZ-ESC-4-p30','DPZ-ESC-4-p33','UKE-ESC-0-p98',]
fib=['DPZ-Fib-0-p06','DPZ-Fib-0-p02','UKE-Fib-0-p03',]
pipsc=['UKE-piPSC-p05-CM','UKE-piPSC-p05-LF','UKE-piPSC-p05-EL','UKE-piPSC-p05-LC']
ipsc=['DPZ-iPSC-1-p18','DPZ-iPSC-1-p19','DPZ-iPSC-1-p22','DPZ-iPSC-1-p24']

#Definition der Farbpaletten
esc_color_red = ['#ffe6e6', '#ffcccc', '#ffb3b3', '#ff9999',
               '#ff8080', '#ff6666', '#ff5050', '#ff4d4d',
               '#ff3333', '#ff1a1a', '#ff0000', '#e60000',
               '#cc0000']
ipsc_color_blue =Pal.Blues5
pipsc_color_green = Pal.Greys4
fib_color_orange = Pal.Oranges4

In [3]:
PORT = PORT
print(PORT)
def make_document(doc):
    
    #Preparing Data
    classes = list(normalized_data)[1:]
    x_data = range(23323)

    p1 = figure(title="Display of Genexpression",
                x_axis_label="Genes",y_axis_label="Expressionswert",width=1000, height=1000)
    
    source_dict=dict(x=x_data)
    raw_dict=dict(x=x_data)
    active_dict=dict(x=x_data)
    log_dict=dict(x=x_data)      
    log_median_dict=dict(x=x_data)
    
    #Generation of linear Data in raw_dict
    for label in classes:
        selected_column = normalized_data[label].sort_values()
        source_dict[label] = selected_column.values.tolist()
        active_dict[label] = selected_column.values.tolist()
        
        selected_column = raw_data[label].sort_values()
        raw_dict[label] = selected_column.values.tolist()

        
    source = ColumnDataSource(source_dict) 
    raw = ColumnDataSource(raw_dict)
    active = ColumnDataSource(active_dict)
      
    #Generate logarithmic and median data   
    data = source.data
    tmp = {} 
    tmp2 = {} 
    for entry in data:
        if entry == 'x':
            tmp[entry] = data[entry]
            tmp2[entry] = data[entry]
        else:
            match = data[entry]
            temp_list = []
            temp_list2 = []
            scm = np.log(np.median(match))
            for i in match:
                if i != 0:
                    temp_list2.append(np.log(i)-scm)
                    temp_list.append(np.log(i))
                else:
                    temp_list.append(i)
                    temp_list2.append(i)
            
            tmp[entry] = temp_list                    
            tmp2[entry] = temp_list2 
       
    log = ColumnDataSource(tmp) 
    log_median = ColumnDataSource(tmp2)
     
    #Zeichnen der Linien
    #Ohne die Variable punkt wird in der Legende die Daten anstatt die Namen angezeigt    
    punkt = "."
    esc_counter = 0
    ipsc_counter = 0
    pipsc_counter = 0
    fib_counter = 0
    for label in classes:
        if label in esc:
            p1.line(x ='x', y=label, source = active, line_color= esc_color_red[esc_counter], name= label+punkt, legend=label+punkt)
            esc_counter = esc_counter + 1
        if label in fib:
            p1.line(x ='x', y=label, source = active, line_color= fib_color_orange[fib_counter], name= label+punkt, legend=label+punkt)
            fib_counter = fib_counter + 1
        if label in pipsc:
            p1.line(x ='x', y=label, source = active, line_color=pipsc_color_green[pipsc_counter], name= label+punkt, legend=label+punkt)
            pipsc_counter = pipsc_counter + 1
        if label in ipsc:
            p1.line(x ='x', y=label, source = active, line_color=ipsc_color_blue[ipsc_counter], name= label+punkt, legend=label+punkt)
            ipsc_counter = ipsc_counter + 1
     
    
    p1.legend.location = "top_left"
    p1.legend.click_policy="hide"    
 
    #handler-Methoden
    def my_button1_handler():            
        active.data = log.data
        print(active==log)
    
    def my_button2_handler():
        active.data = source.data
    
    def my_button3_handler():
        active.data = log_median.data
        
    def my_button4_handler():
        active.data = raw.data
        
    button_list = []
    
    button1 = Button(label = "Logarithmic Representation",button_type="success")
    button2 = Button(label = "Linear Representation",button_type="success")
    button3 = Button(label = "Median (logarithmic Representation)", button_type="success")
    button4 = Button(label = "Raw-Data", button_type="success")
    button_list.append(button1)
    button_list.append(button2)
    button_list.append(button3)
    button_list.append(button4)
    column1 = widgetbox(button_list)
    
    button1.on_click(my_button1_handler)
    button2.on_click(my_button2_handler)
    button3.on_click(my_button3_handler)
    button4.on_click(my_button4_handler)
    layout = row(column1,p1)                   
    doc.add_root(layout)     
apps = {'/': Application(FunctionHandler(make_document))}
server = Server(apps, port=PORT, allow_websocket_origin=['localhost:' + str(PORT), 'localhost:' + str(PORT)])
server.start()  

8000


In [4]:
#helperfunctions for reading in the file with gennames to display
def search(line):
    select = []
    pos = []
    genes =list(normalized_data["gene"]) 
    for i in range(0,len(genes)):
        entry = genes[i]
        if line in entry:
            select.append(entry)
            pos.append(i)
    return (select , pos)

def read_file(filename):
    read = []
    select = []
    pos = []
    result = []
    file = open(filename,"r")
    for line in file:
        line = line.replace('\t','')
        line = line.replace('\n', '')
        if line:
            result ,search_pos = search(line)
            select = select+result
            pos = pos +search_pos
            read.append(line)                     
    file.close()
    return (select,pos)

In [5]:
PORT = PORT+1
print(PORT)
def make_document(doc):
     
    #Extract the genes mentioned in the datafile
    gen_list , indize_list = read_file("pluripotencygenes.txt")
    new_normalized=normalized_data.iloc[indize_list]
    array=new_normalized.as_matrix()
            
    widget_list1 =[]
    text_import = TextInput(value="Data.txt", title="Data:")
    widget_list1.append(text_import)
    
    
    classes = list(normalized_data)[1:]    
    p1 = figure(title="Logarithmic Presentation of Measurements", y_axis_type="log",y_axis_label="Expressionvalues",x_axis_label="Gene", width=2000, height=1000)
    
    #create dictionaries
    esc_dict={}
    ipsc_dict={}   
    pipsc_dict={}
    fib_dict={}
    
    esc_active_dict={}
    ipsc_active_dict={}
    fib_active_dict={}
    pipsc_active_dict={}
    
    gene = list(new_normalized["gene"])

    for i in range(len(gene)):
        #esc
        selected_column = array[i,:13]
        name=selected_column[0]
        values=selected_column[1:]
        esc_dict[name] = values.tolist()
        esc_dict[str(i)] = np.repeat(i,12)

        #ipsc
        selected_column = [array[i,0]]
        zusatz=array[i,13:17].tolist()
        selected_column=selected_column+zusatz
        name=selected_column[0]
        values=selected_column[1:]
        ipsc_dict[name] = values
        ipsc_dict[str(i)] = np.repeat(i,4)
        
        #fib
        selected_column = [array[i,0]]
        zusatz=array[i,17:20].tolist()
        selected_column=selected_column+zusatz
        name=selected_column[0]
        values=selected_column[1:]
        fib_dict[name] = values
        fib_dict[str(i)] = np.repeat(i,3)
        
        #pipsc
        selected_column = [array[i,0]]
        zusatz=array[i,20:].tolist()
        selected_column=selected_column+zusatz
        name=selected_column[0]
        values=selected_column[1:]
        pipsc_dict[name] = values
        pipsc_dict[str(i)] = np.repeat(i,4)
    
   
    esc = ColumnDataSource(esc_dict)
    ipsc = ColumnDataSource(ipsc_dict)   
    fib = ColumnDataSource(fib_dict)
    pipsc = ColumnDataSource(pipsc_dict)   
    
    esc_active = ColumnDataSource(esc_dict)
    ipsc_active = ColumnDataSource(ipsc_dict)
    fib_active = ColumnDataSource(fib_dict)
    pipsc_active = ColumnDataSource(pipsc_dict)
    
    esc_active.data=esc.data
    ipsc_active.data=ipsc.data
    fib_active.data=fib.data
    pipsc_active.data=pipsc.data
    
    #print(esc_active.data)
    #print(esc.data)
    
    #draw lines
    counter = 1
    x_counter=0
    for gen_name in gene:
        
        p1.diamond(x =str(x_counter), y=gen_name, source = esc_active, line_color= "red", name= gen_name+str(counter), legend=gen_name+str(counter),size=14)

        p1.diamond(x =str(x_counter), y=gen_name, source = ipsc_active, line_color="yellow" , name= gen_name+str(counter), legend=gen_name+str(counter),size=14)

        p1.diamond(x =str(x_counter), y=gen_name, source = fib_active, line_color="blue", name= gen_name+str(counter), legend=gen_name+str(counter),size=14)

        p1.diamond(x =str(x_counter), y=gen_name, source = pipsc_active, line_color="black", name= gen_name+str(counter), legend=gen_name+str(counter),size=14)
        x_counter+=1
        counter+=1
        
    def text_import_handler(attr, old, new):
        global gen_list
        global indize_list
                           
        gen_list , indize_list = read_file(new)
        new_normalized=normalized_data.iloc[indize_list]
        array=new_normalized.as_matrix()
  
        esc_dict={}
        ipsc_dict={}   
        pipsc_dict={}
        fib_dict={}

        gene = list(new_normalized["gene"])

        for i in range(len(gene)):
            #esc
            selected_column = array[i,:13]
            name=selected_column[0]
            values=selected_column[1:]
            esc_dict[name] = values.tolist()
            esc_dict[str(i)] = np.repeat(i,12)

            #ipsc
            selected_column = [array[i,0]]
            zusatz=array[i,13:17].tolist()
            selected_column=selected_column+zusatz
            name=selected_column[0]
            values=selected_column[1:]
            ipsc_dict[name] = values
            ipsc_dict[str(i)] = np.repeat(i,4)

            #fib
            selected_column = [array[i,0]]
            zusatz=array[i,17:20].tolist()
            selected_column=selected_column+zusatz
            name=selected_column[0]
            values=selected_column[1:]
            fib_dict[name] = values
            fib_dict[str(i)] = np.repeat(i,3)

            #pipsc
            selected_column = [array[i,0]]
            zusatz=array[i,20:].tolist()
            selected_column=selected_column+zusatz
            name=selected_column[0]
            values=selected_column[1:]
            pipsc_dict[name] = values
            pipsc_dict[str(i)] = np.repeat(i,4)
    
        esc = ColumnDataSource(esc_dict)
        ipsc = ColumnDataSource(ipsc_dict)   
        fib = ColumnDataSource(fib_dict)
        pipsc = ColumnDataSource(pipsc_dict)
                
        
        #Bug: Dictionary esc_Active is changed, but the plot does not get updated
        #print(esc_active.data)
        esc_active.data=esc.data
        ipsc_active.data=ipsc.data
        fib_active.data=fib.data
        pipsc_active.data=pipsc.data        
        #print(esc_active.data)        
        #print(esc.data)
        
    text_import.on_change("value", text_import_handler)
    
    p1.legend.location = "top_left"
    p1.legend.click_policy="hide"
    
    layout2 = row(widget_list1)       
    layout = row(p1) 
    doc.add_root(layout2)
    doc.add_root(layout)     
apps = {'/': Application(FunctionHandler(make_document))}
server = Server(apps, port=PORT, allow_websocket_origin=['localhost:' + str(PORT), 'localhost:' + str(PORT)])
server.start()  

8001


In [6]:
interesting_genes = []
klasse1= []
klasse2 = []
gene = []

def interesting_genes(ausgewählte_klassen1, ausgewählte_klassen2):
    global klasse1
    global klasse2
    global gene
    
    #Map buttons to classes
    samples = []
    for i in range(len(ausgewählte_klassen1)):
        if (ausgewählte_klassen1[i]==0):
            samples = samples + esc
        if (ausgewählte_klassen1[i]==3):
            samples = samples + fib 
        if (ausgewählte_klassen1[i]==2):
            samples = samples + pipsc 
        if (ausgewählte_klassen1[i]==1):
            samples = samples + ipsc  
    klasse1 = normalized_data[samples]
    
    samples = []    
    for i in range(len(ausgewählte_klassen2)):
        if (ausgewählte_klassen2[i]==0):
            samples = samples + esc
        if (ausgewählte_klassen2[i]==3):
            samples = samples + fib 
        if (ausgewählte_klassen2[i]==2):
            samples = samples + pipsc 
        if (ausgewählte_klassen2[i]==1):
            samples = samples + ipsc           
    klasse2 = normalized_data[samples]

    #Calculate interesting genes
    for i in range(gen_nr):
        class1 = pd.Series(klasse1.iloc[i] )
        class2 = pd.Series(klasse2.iloc[i])

        mean1 = class1.mean()
        mean2 = class2.mean()
        if mean1 < mean2:
            maxx= class1.max()
            minn = class2.min()
        else:
            minn = class1.min()
            maxx = class2.max()
        if minn > maxx:
            interesting_genes.append(i) 
            
    #Bildung von Tupeln aus Namen und Index
    for gen_id in interesting_genes:
        name = normalized_data["gene"].iloc[gen_id]
        gene.append((gen_id,name))
    return (gene, klasse1, klasse2)    

In [7]:
PORT = PORT+1
print(PORT)
def make_document(doc):

    button_list = []

    p = figure(width=1500, height = 1000, title = "Aufgabe 3" , x_axis_label = "Gen Expression", y_axis_label="")
     
    check_button_group = CheckboxButtonGroup(
    labels=["ESC","IPSC","PIPSC","FIB"], active = [1])
    check_button_group1 = CheckboxButtonGroup(
    labels=["ESC","IPSC","PIPSC","FIB"], active = [1])
    
    button = Button(label="Show Genes", button_type="success")
    
    select = Select(title="Gen:", value="Gene", options=["-"],width=200)   

    button_list.append(check_button_group)
    button_list.append(check_button_group1)
    button_list.append(button)    
    button_list.append(select)
    
    active_dict=dict()
    active = ColumnDataSource(active_dict)
    active2_dict=dict()
    active2 = ColumnDataSource(active2_dict)
    
    
    def my_select_handler(attr,old,new): 
    #Erstellung der Dictionarys mit den Daten des ausgewählten Gens
        for gen in gene:
            if select.value == gen[1]:            
                y1 = klasse1.iloc[gen[0]] 
                y2 = klasse2.iloc[gen[0]]
           
        source_dict=dict(x=np.repeat(1,len(list(y1))),y=y1) 
        source = ColumnDataSource(source_dict)
        source2_dict=dict(x=np.repeat(1,len(list(y2))),y=y2) 
        source2 = ColumnDataSource(source2_dict)
            
        active.data=source.data
        active2.data=source2.data
       
        #Zeichnen der Linien
        line1 = p.diamond('y','x' ,source=active, legend="Class 1",color="red",size=10)
        line2 = p.diamond('y','x' ,source=active2, legend="Class 2",color="blue",size=10)                                               

    def my_click_handler():
        gene, klasse1, klasse2 = interesting_genes(check_button_group.active,check_button_group1.active)
        #Darstellung der interessanten Gene in der Select-Box   
        _, list2 = zip(*gene)
        select.options= list(list2)            
  
    select.on_change("value",my_select_handler)   
    button.on_click(my_click_handler)

    column1 = widgetbox(button_list)
    
    layout = row(column1,p) 
    doc.add_root(layout)     
apps = {'/': Application(FunctionHandler(make_document))}
server = Server(apps, port=PORT, allow_websocket_origin=['localhost:' + str(PORT), 'localhost:' + str(PORT)])
server.start()

8002


ERROR:bokeh.server.protocol_handler:error handling message Message 'PATCH-DOC' (revision 1) content: {'events': [{'kind': 'ModelChanged', 'model': {'type': 'Button', 'id': '1744'}, 'attr': 'clicks', 'new': 1}], 'references': []}: AttributeError("'function' object has no attribute 'append'",)
/Users/niclas/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
ERROR:bokeh.server.protocol_handler:error handling message Message 'PATCH-DOC' (revision 1) content: {'events': [{'kind': 'ModelChanged', 'model': {'type': 'Button', 'id': '1744'}, 'attr': 'clicks', 'new': 2}], 'references': []}: AttributeError("'function' object has no attribute 'append'",)
/Users/niclas/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:106: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
ERROR:bokeh.server.protocol_handler:error handling message Message 'PAT